# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("marketing_customer_analysis_clean.csv")
df.columns


Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type', 'month'],
      dtype='object')

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [4]:
df2 = df[(df["total_claim_amount"] > 1000) & (df["response"] == 'Yes')] #here, i applied the two filters, we must write & (not and)

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [5]:
responded_yes = df[df["response"] == "Yes"]
responded_yes.pivot_table(
    values = "total_claim_amount",
    index = ["policy_type", "gender"],
    aggfunc = "mean"
    ).round(2)

total_claim_amount
policy_type    gender                    
Corporate Auto F                   433.74
               M                   408.58
Personal Auto  F                   452.97
               M                   457.01
Special Auto   F                   453.28
               M                   429.53

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [6]:
a = df.groupby("state")["customer"].count().reset_index()
more_than_500 = a[a["customer"] > 500] 
a

,state,customer
0,Arizona,1937
1,California,4183
2,Nevada,993
3,Oregon,2909
4,Washington,888


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [7]:
df[["customer_lifetime_value", "education", "gender"]].groupby(by=["gender", "education"]).agg({"median"}).round(2)

customer_lifetime_value
                                             median
gender education                                   
F      Bachelor                             5640.51
       College                              5623.61
       Doctor                               5332.46
       High School or Below                 6039.55
       Master                               5729.86
M      Bachelor                             5548.03
       College                              6005.85
       Doctor                               5577.67
       High School or Below                 6286.73
       Master                               5579.10

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [8]:
df.columns

Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type', 'month'],
      dtype='object')

In [9]:
df[["number_of_policies", "state", "month"]].groupby(by= ["state", "month"]).agg("count")

number_of_policies
state      month                    
Arizona    1                    1008
           2                     929
California 1                    2231
           2                    1952
Nevada     1                     551
           2                     442
Oregon     1                    1565
           2                    1344
Washington 1                     463
           2                     425

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [10]:
df2 = df[["number_of_policies", "state", "month"]].groupby(by= ["state", "month"]).agg("count").sort_values(by=["number_of_policies"], ascending=False).reset_index()
print(df2)

        state  month  number_of_policies
0  California      1                2231
1  California      2                1952
2      Oregon      1                1565
3      Oregon      2                1344
4     Arizona      1                1008
5     Arizona      2                 929
6      Nevada      1                 551
7  Washington      1                 463
8      Nevada      2                 442
9  Washington      2                 425


In [11]:
df2[["state", "number_of_policies"]].groupby(by=["state"]).agg("sum").sort_values(by=["number_of_policies"], ascending=False)


,number_of_policies
state,
California,4183
Oregon,2909
Arizona,1937
Nevada,993
Washington,888


In [15]:
top_states = df2["state"].unique()[:3]

new_df = df2[df2["state"].isin(top_states)]
new_df

,state,month,number_of_policies
0,California,1,2231
1,California,2,1952
2,Oregon,1,1565
3,Oregon,2,1344
4,Arizona,1,1008
5,Arizona,2,929


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [21]:
df2 = df[df["response"] == "Yes"].reset_index()
df2[["sales_channel", "response"]].groupby(by="sales_channel").agg("sum")

response_rate_table = df2.groupby('sales_channel')['response'].count().reset_index()
response_rate_table
response_rate_table["rate"] = response_rate_table["response"] / df2.shape[0] * 100
